In [ ]:
%set_env PYTORCH_ENABLE_MPS_FALLBACK=1

env: PYTORCH_ENABLE_MPS_FALLBACK=1


In [ ]:
#| default_exp models.tkan

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# TKAN

In [ ]:
#| hide
from fastcore.test import test_eq
from nbdev.showdoc import show_doc

In [ ]:
#| export
# from typing import Optional

import torch
import torch.nn as nn
import torch.nn.functional as F

# from neuralforecast.losses.pytorch import MAE
# from neuralforecast.common._base_windows import BaseWindows

In [ ]:
#| export

class FixedSplineActivation(nn.Module):
    def __init__(self, 
                 exponent: float = 1.0, 
                 max_exponent: 
                 float = 9.0) -> None:
        
        super(FixedSplineActivation, self).__init__()
        
        self.exponent = torch.tensor(exponent, dtype=torch.float32)
        self.max_exponent = torch.tensor(max_exponent, dtype=torch.float32)
        self.epsilon = torch.finfo(torch.float32).eps
        self.exponent = torch.clamp(self.exponent, -self.max_exponent, self.max_exponent)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        inputs_safe = torch.clamp(inputs, self.epsilon, 1.0)
        return torch.pow(inputs_safe, self.exponent)

In [ ]:
#| export

class PowerSplineActivation(nn.Module):
    def __init__(self, 
                 initial_exponent: float = 1.0, 
                 epsilon: float = 1e-7, 
                 max_exponent: float = 9.0, 
                 trainable: bool = True) -> None:
        
        super(PowerSplineActivation, self).__init__()

        self.initial_exponent = torch.tensor(initial_exponent, dtype=torch.float32)
        self.epsilon = epsilon
        self.max_exponent = torch.tensor(max_exponent, dtype=torch.float32)
        self.trainable = trainable

        if self.trainable:
            self.exponent = nn.Parameter(torch.tensor(self.initial_exponent, dtype=torch.float32))
            self.bias = nn.Parameter(torch.zeros(1, dtype=torch.float32))
        else:
            self.exponent = torch.tensor(self.initial_exponent, dtype=torch.float32)
            self.bias = torch.zeros(1, dtype=torch.float32)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        clipped_exponent = torch.clamp(self.exponent, -self.max_exponent, self.max_exponent)
        x_safe = torch.clamp(inputs + self.bias, self.epsilon, 1.0)
        return torch.pow(x_safe, clipped_exponent)

In [ ]:
#| export

class LinspaceInitializer:
    def __init__(self, start: float, stop: float, num: int):
        self.start = start
        self.stop = stop
        self.num = num

    def __call__(self, shape, dtype=torch.float32):
        if torch.prod(torch.tensor(shape)) != self.num:
            raise ValueError(f"The shape {shape} does not match the number of elements {self.num}.")
        result = torch.linspace(self.start, self.stop, self.num, dtype=dtype)
        return result.view(shape)

class BSplineActivation(nn.Module):
    def __init__(self, 
                 num_bases: int = 10, 
                 order: int = 3):
        
        super(BSplineActivation, self).__init__()

        self.num_bases = num_bases
        self.order = order
        self.w = nn.Parameter(torch.nn.init.xavier_uniform_(torch.empty(1)))
        
        self.coefficients = nn.Parameter(torch.zeros(self.num_bases))
        self.bases = nn.Parameter(LinspaceInitializer(0.0, 1.0, self.num_bases)((self.num_bases,)), requires_grad=False)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        x = inputs
        silu = x * torch.sigmoid(x)  # SiLU activation
        spline_output = self.compute_spline(x)
        return self.w * (silu + spline_output)

    def compute_spline(self, x: torch.Tensor) -> torch.Tensor:
        safe_x = torch.clamp(x, torch.finfo(torch.float32).eps, 1.0)
        expanded_x = safe_x.unsqueeze(-1)
        basis_function_values = torch.pow(expanded_x - self.bases, self.order)
        spline_values = torch.sum(self.coefficients * basis_function_values, dim=-1)
        return spline_values

In [ ]:
#| export

class DropoutRNNCell:

    def __init__(self, dropout=0.0, recurrent_dropout=0.0, seed=None):
        self.dropout = dropout
        self.recurrent_dropout = recurrent_dropout
        self.seed = seed
        self._dropout_mask = None
        self._recurrent_dropout_mask = None

    def get_dropout_mask(self, step_input):
        if self._dropout_mask is None and self.dropout > 0:
            self._dropout_mask = F.dropout(torch.ones_like(step_input), p=self.dropout, training=True, inplace=False)
        return self._dropout_mask

    def get_recurrent_dropout_mask(self, step_input):
        if self._recurrent_dropout_mask is None and self.recurrent_dropout > 0:
            self._recurrent_dropout_mask = F.dropout(torch.ones_like(step_input), p=self.recurrent_dropout, training=True, inplace=False)
        return self._recurrent_dropout_mask

    def reset_dropout_mask(self):
        """Reset the cached dropout mask if any."""
        self._dropout_mask = None

    def reset_recurrent_dropout_mask(self):
        self._recurrent_dropout_mask = None

class DropoutLSTMCell(nn.LSTMCell, DropoutRNNCell):
    """
    LSTM cell with dropout applied to inputs and recurrent state.
    """
    def __init__(self, input_size, hidden_size, dropout=0.0, recurrent_dropout=0.0, seed=None):
        nn.LSTMCell.__init__(self, input_size, hidden_size)
        DropoutRNNCell.__init__(self, dropout, recurrent_dropout, seed)

    def forward(self, input, hx=None):
        self.reset_dropout_mask()
        self.reset_recurrent_dropout_mask()
        if self.training:
            input = input * self.get_dropout_mask(input)
        hx = tuple(h * self.get_recurrent_dropout_mask(h) for h in hx) if hx is not None else hx
        return super(DropoutLSTMCell, self).forward(input, hx)


In [ ]:
#| export

class TKANCell(nn.Module, DropoutRNNCell):
    def __init__(
        self,
        units,
        tkan_activations=None,
        activation="tanh",
        recurrent_activation="sigmoid",
        use_bias=True,
        kernel_initializer="glorot_uniform",
        recurrent_initializer="orthogonal",
        bias_initializer="zeros",
        unit_forget_bias=True,
        kernel_regularizer=None,
        recurrent_regularizer=None,
        bias_regularizer=None,
        kernel_constraint=None,
        recurrent_constraint=None,
        bias_constraint=None,
        dropout=0.0,
        recurrent_dropout=0.0,
        seed=None,
    ):
        nn.Module.__init__(self)
        DropoutRNNCell.__init__(self, dropout, recurrent_dropout, seed)

        if units <= 0:
            raise ValueError(
                "Received an invalid value for argument `units`, "
                f"expected a positive integer, got {units}."
            )

        self.units = units
        self.activation = activation
        self.recurrent_activation = recurrent_activation
        self.use_bias = use_bias

        self.kernel_initializer = kernel_initializer
        self.recurrent_initializer = recurrent_initializer
        self.bias_initializer = bias_initializer

        self.kernel_regularizer = kernel_regularizer
        self.recurrent_regularizer = recurrent_regularizer
        self.bias_regularizer = bias_regularizer

        self.kernel_constraint = kernel_constraint
        self.recurrent_constraint = recurrent_constraint
        self.bias_constraint = bias_constraint

        self.unit_forget_bias = unit_forget_bias

        self.tkan_activations = tkan_activations or [BSplineActivation(3)]
        self.tkan_sub_layers = nn.ModuleList()
        
        for act in self.tkan_activations:
            if act is None:
                self.tkan_sub_layers.append(nn.Linear(1, 1, bias=False))
            elif isinstance(act, (int, float)):
                self.tkan_sub_layers.append(nn.Linear(1, 1, bias=False))
            else:
                self.tkan_sub_layers.append(nn.Linear(1, 1, bias=False))

        self.state_size = [units, units] + [1 for _ in self.tkan_activations]
        self.output_size = units

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = nn.Parameter(torch.empty((input_dim, self.units * 3)))
        nn.init.xavier_uniform_(self.kernel)
        self.recurrent_kernel = nn.Parameter(torch.empty((self.units, self.units * 3)))
        nn.init.orthogonal_(self.recurrent_kernel)
        self.sub_tkan_kernel = nn.Parameter(torch.empty((len(self.tkan_sub_layers), 2)))
        nn.init.orthogonal_(self.sub_tkan_kernel)
        self.sub_tkan_recurrent_kernel = nn.Parameter(torch.empty((len(self.tkan_sub_layers), input_shape[1] * 2)))
        nn.init.orthogonal_(self.sub_tkan_recurrent_kernel)
        self.aggregated_weight = nn.Parameter(torch.empty((len(self.tkan_sub_layers), self.units)))
        nn.init.xavier_uniform_(self.aggregated_weight)
        self.aggregated_bias = nn.Parameter(torch.zeros((self.units,)))
        
        if self.use_bias:
            if self.unit_forget_bias:
                self.bias = nn.Parameter(torch.cat([torch.zeros(self.units), torch.ones(self.units), torch.zeros(self.units)]))
            else:
                self.bias = nn.Parameter(torch.zeros((self.units * 3,)))
        else:
            self.bias = None

    def forward(self, inputs, states):
        self.reset_dropout_mask()
        self.reset_recurrent_dropout_mask()
        h_tm1, c_tm1 = states[:2]
        sub_states = states[2:]

        if self.training and self.dropout > 0.0:
            inputs = inputs * self.get_dropout_mask(inputs)
        if self.training and self.recurrent_dropout > 0.0:
            h_tm1 = h_tm1 * self.get_recurrent_dropout_mask(h_tm1)

        sub_outputs = []
        new_sub_states = []

        if self.use_bias:
            x_i, x_f, x_c = torch.split(self.recurrent_activation(F.linear(inputs, self.kernel) + F.linear(h_tm1, self.recurrent_kernel) + self.bias), self.units, dim=-1)
        else:
            x_i, x_f, x_c = torch.split(self.recurrent_activation(F.linear(inputs, self.kernel) + F.linear(h_tm1, self.recurrent_kernel)), self.units, dim=-1)

        for idx, (sub_layer, sub_state) in enumerate(zip(self.tkan_sub_layers, sub_states)):
            sub_kernel_h, sub_kernel_x = torch.split(self.sub_tkan_recurrent_kernel[idx], 2)
            agg_input = inputs * sub_kernel_x + sub_state * sub_kernel_h
            sub_output = sub_layer(agg_input)
            sub_recurrent_kernel_h, sub_recurrent_kernel_x = torch.split(self.sub_tkan_kernel[idx], 2)
            new_sub_state = sub_recurrent_kernel_h * sub_output + sub_state * sub_recurrent_kernel_x

            sub_outputs.append(sub_output)
            new_sub_states.append(new_sub_state)

        sub_outputs = torch.stack(sub_outputs)
        aggregated_sub_output = sub_outputs.view(inputs.size(0), -1)
        aggregated_input = torch.matmul(aggregated_sub_output, self.aggregated_weight) + self.aggregated_bias

        xo = self.recurrent_activation(aggregated_input)

        c = x_f * c_tm1 + x_i * x_c
        h = xo * self.activation(c)

        new_states = [h, c] + new_sub_states
        return h, new_states

    def get_initial_state(self, batch_size, dtype=torch.float32):
        return [torch.zeros((batch_size, self.units), dtype=dtype),
                torch.zeros((batch_size, self.units), dtype=dtype)] + [torch.zeros((batch_size, 1), dtype=dtype) for _ in range(len(self.tkan_sub_layers))]
